### Agent-Lab: Coordinator -> Planner -> Supervisor Agent

Objective of this notebook is evaluating and adapting a [Multi-Agent Supervisor Architecture](https://langchain-ai.github.io/langgraph/concepts/multi_agent/#supervisor) with coordinator and execution planning steps.

#### Preparation steps:

Before executing the notebook perform the following preparation steps.

1. Start Docker containers: `docker compose up -d --build`

2. Verify application is up and running, with web browser: `http://localhost:18000/docs`

3. Inside project root directory, create a `.env` file with env vars with correct values:

    ```
    DEVELOPING="True"
    OLLAMA_ENDPOINT="http://localhost:11434"
    TAVILY_API_KEY="???"
    XAI_API_KEY="???"
    ```

---

In [ ]:
from browser_use import Agent, Browser, ChatOpenAI
import asyncio

# Connect to your existing Chrome browser
browser = Browser(
    cdp_url='ws://192.168.2.195:9222'
)

agent = Agent(
    task='Visit https://duckduckgo.com and search for "browser-use founders"',
    browser=browser,
    llm=ChatOpenAI(model='gpt-4.1-mini'),
)

async def run_agent():
    await agent.run()

asyncio.run(run_agent())

In [ ]:
%%capture

import os

os.chdir("..")

from dotenv import load_dotenv

load_dotenv()

from IPython.display import Image, Markdown, display

from notebooks import experiment_utils
from app.core.container import Container
from app.interface.api.messages.schema import MessageRequest

# graph rendering
import nest_asyncio

nest_asyncio.apply()

# start dependency injection container
container = Container()
container.init_resources()
container.wire(modules=[__name__])

In [ ]:
# create agent
# agent = experiment_utils.create_ollama_agent(
#     agent_type="coordinator_planner_supervisor",
#     llm_tag="granite3.2:latest",
#     ollama_endpoint=os.getenv("OLLAMA_ENDPOINT")
# )

# vision model is required by browser-lab (uses vision to parse the rendered page)
agent = experiment_utils.create_xai_agent(
    agent_type="coordinator_planner_supervisor",
    llm_tag="grok-2-vision",
    api_key=os.getenv("XAI_API_KEY"),
)

# updated_agent = experiment_utils.update_agent_setting(
#     agent_id=agent["id"],
#     setting_key="deep_search_mode",
#     setting_value="True"
# )

In [ ]:
# get agent instance
coordinator_planner_supervisor_agent = container.coordinator_planner_supervisor_agent()

# get checkpointer instance
graph_persistence_factory = container.graph_persistence_factory()
checkpointer = graph_persistence_factory.build_checkpoint_saver()

# get config
agent_config = coordinator_planner_supervisor_agent.get_config(agent["id"])

# Create Graph
workflow_builder = coordinator_planner_supervisor_agent.get_workflow_builder(
    agent["id"]
)
workflow = workflow_builder.compile(checkpointer=checkpointer)
experiment_utils.print_graph(workflow)

### Evaluate Researcher

In [ ]:
%%capture
message = MessageRequest(
    message_role="human",
    message_content="What characterizes a good general according to Sun Tzu?",
    agent_id=agent["id"],
)

inputs = coordinator_planner_supervisor_agent.get_input_params(message)
result = workflow.invoke(inputs, agent_config)
ai_message_content, workflow_state = (
    coordinator_planner_supervisor_agent.format_response(result)
)

In [ ]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

---
### Evaluate Coder

In [ ]:
%%capture
message = MessageRequest(
    message_role="human",
    message_content="With coder, generate a simple hello world in Python.",
    agent_id=agent["id"],
)

inputs = coordinator_planner_supervisor_agent.get_input_params(message)
result = workflow.invoke(inputs, agent_config)
ai_message_content, workflow_state = (
    coordinator_planner_supervisor_agent.format_response(result)
)

In [ ]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

#### Evaluate coder memory (convert function to other language)

In [ ]:
%%capture
message = MessageRequest(
    message_role="human",
    message_content="Please convert this python code to bash.",
    agent_id=agent["id"],
)

inputs = coordinator_planner_supervisor_agent.get_input_params(message)
result = workflow.invoke(inputs, agent_config)
ai_message_content, workflow_state = (
    coordinator_planner_supervisor_agent.format_response(result)
)

In [ ]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

---
### Evaluate Browser

In [ ]:
# %%capture
message = MessageRequest(
    message_role="human",
    message_content=(
        "With the browser, go to https://en.wikipedia.org/wiki/Mathematical_finance and summarize the article."
    ),
    agent_id=agent["id"],
)

inputs = coordinator_planner_supervisor_agent.get_input_params(message)
result = workflow.invoke(inputs, agent_config)
ai_message_content, workflow_state = (
    coordinator_planner_supervisor_agent.format_response(result)
)

In [ ]:
display(Markdown("**Browser Navigation:**"))
display(Image(filename="notebooks/coordinator_planner_supervisor_agent.gif"))

In [ ]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))